In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

np.random.seed(2)

In [21]:
data = pd.read_csv('cred-data.csv')

## Data Exploration

In [3]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Data Pre-Processing

In [4]:
data.drop('Time', axis=1, inplace=True)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [6]:
data['NormalisedAmount'] = scaler.fit_transform(data[['Amount']])
data.drop('Amount', axis=1, inplace=True)

In [7]:
X = data.drop('Class', axis=1)
y = data['Class']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

In [9]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [10]:
(X_train.shape, y_train.shape), (X_test.shape,y_test.shape)

(((199364, 29), (199364,)), ((85443, 29), (85443,)))

## Building the Nueral Network

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [12]:
model = Sequential()

model.add(Dense(units=16, activation='relu', input_dim = 29))
model.add(Dense(units=24, activation='relu'))

model.add(Dropout(0.3))

model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=24, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                480       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                408       
_________________________________________________________________
dropout (Dropout)            (None, 24)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                500       
_________________________________________________________________
dense_3 (Dense)              (None, 24)                504       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 25        
Total params: 1,917
Trainable params: 1,917
Non-trainable params: 0
______________________________________________________

## Training the Network

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
train_history = model.fit(x=X_train, y=y_train, epochs=5, batch_size=15, verbose=1, validation_data=(X_test,y_test))

Epoch 1/5
13291/13291 [==============================] - 22s 2ms/step - loss: 0.0072 - accuracy: 0.9991 - val_loss: 0.0035 - val_accuracy: 0.9994
Epoch 2/5
13291/13291 [==============================] - 21s 2ms/step - loss: 0.0036 - accuracy: 0.9994 - val_loss: 0.0038 - val_accuracy: 0.9994
Epoch 3/5
13291/13291 [==============================] - 21s 2ms/step - loss: 0.0032 - accuracy: 0.9994 - val_loss: 0.0036 - val_accuracy: 0.9994
Epoch 4/5
13291/13291 [==============================] - 21s 2ms/step - loss: 0.0032 - accuracy: 0.9994 - val_loss: 0.0044 - val_accuracy: 0.9994
Epoch 5/5
13291/13291 [==============================] - 25s 2ms/step - loss: 0.0031 - accuracy: 0.9994 - val_loss: 0.0040 - val_accuracy: 0.9994


In [16]:
y_pred = model.predict(X_test)

In [17]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [18]:
confusion_matrix(y_test, y_pred.round())

array([[85272,    24],
       [   28,   119]], dtype=int64)

In [20]:
model.save('cred_fraud_detection')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: cred_fraud_detection\assets
